In [50]:
import pandas as pd
import numpy as np
from transformers import pipeline
from transformers import RobertaTokenizer, RobertaModel, AutoModel

In [4]:
all_data = pd.read_csv("vaalit_2019.csv")

In [18]:
print(all_data.info())

# Look at lappi related columns
lappi_columns = [col for col in all_data.columns if col[:6] == 'Lappi.']
print(lappi_columns, len(lappi_columns))

# Print answers to a lappi question. We'll use these to test the encoder.
lappi_question_idx = -1
lappi_data = all_data[lappi_columns[lappi_question_idx]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2437 entries, 0 to 2436
Columns: 212 entries, vaalipiiri to Lappi. Jäämeren rata pitää rakentaa..1
dtypes: float64(5), int64(1), object(206)
memory usage: 3.9+ MB
None
['Lappi. Lappiin ei saa avata yhtään uutta kaivosta ennen kuin yhtiöiltä aletaan periä kaivosveroa.', 'Lappi. Mahdollisen uuden maakuntahallinnon toimintoja ja työpaikkoja on sijoitettava muuallekin kuin Rovaniemelle.', 'Lappi. Lapin erikoissairaanhoito on keskitettävä kokonaan Rovaniemelle Lapin keskussairaalaan.', 'Lappi. Alkuperäiskansojen oikeuksia koskeva ILO-sopimus pitää ratifioida seuraavalla hallituskaudella. ', 'Lappi. Jäämeren rata pitää rakentaa.', 'Lappi. Lappiin ei saa avata yhtään uutta kaivosta ennen kuin yhtiöiltä aletaan periä kaivosveroa..1', 'Lappi. Mahdollisen uuden maakuntahallinnon toimintoja ja työpaikkoja on sijoitettava muuallekin kuin Rovaniemelle..1', 'Lappi. Lapin erikoissairaanhoito on keskitettävä kokonaan Rovaniemelle Lapin keskussairaalaan

In [33]:
lappi_data.iloc[2432]

'Nykyinen ratasuunnitelma on todettu tutkimuksissa nykytilanteessa kannattamattomaksi. Lisäksi hanke aiheuttaa ongelmia ympäristölle ja alueen väestölle. Suunnitelmassa on kuitenkin yksi mielenkiintoinen osa. Junaradan jatkaminen Kemijärveltä Pyhätunturin kautta Sodankylään voisi olla toteuttamiskelpoinen ajatus.\n\nRatayhteys Jäämerelle voidaan saada aikaan toisella tavalla varsin helposti ja selvästi halvemmalla. Päivitetään nykyinen  Laurila-Tornio-Haaparanta ratayhteys. Tämän päivityksen hinta on vain murto-osa nyt suunnitellusta linjauksesta ja sillä saavutettaisiin ratayhteys Jäämeren ja Atlantin lisäksi myös keskeiseen Eurooppaan ja vaikka Välimerelle saakka. '

In [36]:
print(lappi_data.iloc[2432])

Nykyinen ratasuunnitelma on todettu tutkimuksissa nykytilanteessa kannattamattomaksi. Lisäksi hanke aiheuttaa ongelmia ympäristölle ja alueen väestölle. Suunnitelmassa on kuitenkin yksi mielenkiintoinen osa. Junaradan jatkaminen Kemijärveltä Pyhätunturin kautta Sodankylään voisi olla toteuttamiskelpoinen ajatus.

Ratayhteys Jäämerelle voidaan saada aikaan toisella tavalla varsin helposti ja selvästi halvemmalla. Päivitetään nykyinen  Laurila-Tornio-Haaparanta ratayhteys. Tämän päivityksen hinta on vain murto-osa nyt suunnitellusta linjauksesta ja sillä saavutettaisiin ratayhteys Jäämeren ja Atlantin lisäksi myös keskeiseen Eurooppaan ja vaikka Välimerelle saakka. 


In [58]:
tokenizer = RobertaTokenizer.from_pretrained('Finnish-NLP/roberta-large-finnish')
# model = RobertaModel.from_pretrained('Finnish-NLP/roberta-large-finnish')
model_roberta = AutoModel.from_pretrained('Finnish-NLP/roberta-large-finnish')
model_turku = AutoModel.from_pretrained("TurkuNLP/bert-large-finnish-cased-v1")

models = [model_roberta, model_turku]

Some weights of the model checkpoint at Finnish-NLP/roberta-large-finnish were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at Finnish-NLP/roberta-large-finnish and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

In [60]:
# models[0]
models[1]

(RobertaModel(
   (embeddings): RobertaEmbeddings(
     (word_embeddings): Embedding(50265, 1024, padding_idx=1)
     (position_embeddings): Embedding(514, 1024, padding_idx=1)
     (token_type_embeddings): Embedding(1, 1024)
     (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
     (dropout): Dropout(p=0.1, inplace=False)
   )
   (encoder): RobertaEncoder(
     (layer): ModuleList(
       (0): RobertaLayer(
         (attention): RobertaAttention(
           (self): RobertaSelfAttention(
             (query): Linear(in_features=1024, out_features=1024, bias=True)
             (key): Linear(in_features=1024, out_features=1024, bias=True)
             (value): Linear(in_features=1024, out_features=1024, bias=True)
             (dropout): Dropout(p=0.1, inplace=False)
           )
           (output): RobertaSelfOutput(
             (dense): Linear(in_features=1024, out_features=1024, bias=True)
             (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine

In [55]:
text = lappi_data.iloc[2432]
tokenized_input = tokenizer(text, return_tensors='pt')
token_encodings = model_roberta(**tokenized_input)

print(output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.9925, -0.2804,  0.0515,  ...,  0.9504, -0.0999,  0.6405],
         [ 1.7736,  0.1815, -0.8647,  ...,  0.8231,  0.2149,  0.7865],
         [ 0.2692, -0.4999,  0.3979,  ...,  0.3658,  0.0418,  0.9323],
         ...,
         [ 0.5091,  0.3476, -0.2135,  ...,  0.5232,  0.2879, -0.9866],
         [ 1.8058,  0.2573,  0.4596,  ...,  0.0110,  0.3241, -1.6292],
         [ 0.7662, -0.6481,  0.1958,  ..., -0.0296, -0.3188,  1.5189]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.2203, -0.5763, -0.2142,  ...,  0.2957,  0.2482, -0.2921]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)


In [56]:
# Average pool the outputs into a single feature vector
print(token_encodings['input_ids'].shape) # B, T
print(token_encodings[0].shape) # B, T, D

encoded


KeyError: 'input_ids'